In [1]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
LowerGameBound = 100

In [2]:
def cleanGL(x):
    try:
        out = float(x)
    except:
        out = 0
    return out

In [3]:
CurrentPitcherRoster = pd.read_csv('../Data/DataBaseCrawler/MLBPitcherRoster2018.csv', index_col=0)
CurrentPitcherRoster = CurrentPitcherRoster[['Name','Team','ID']]
CurrentPitcherRoster.shape

(766, 3)

In [4]:
AggregatedPitcherDataFrame=pd.read_csv('../Data/DataBaseCrawler/GameLogs/InactivePitcherList.csv')
AggregatedPitcherDataFrame.drop_duplicates(subset=['ID'], inplace=True)
AggregatedPitcherDataFrame = AggregatedPitcherDataFrame[~AggregatedPitcherDataFrame['ID'].isin(CurrentPitcherRoster['ID'])]

In [5]:
ID = CurrentPitcherRoster['ID'][0]
GameLog = pd.read_csv('../Data/DataBaseCrawler/GameLogs/Pitchers/'+str(ID)+'GL.csv',index_col=0)
#GameLog=GameLog.drop(['GS_x','GSv2'], axis=1)

#Grab the splitLogs
SplitLogHand = pd.read_csv('../Data/DataBaseCrawler/GameLogs/Pitchers/'+str(ID)+'HandSL.csv', index_col=0)
SplitLogHomeAway = pd.read_csv('../Data/DataBaseCrawler/GameLogs/Pitchers/'+str(ID)+'HomeAwaySL.csv')

In [6]:
cleanRowNorm = [x for x in map(cleanGL,GameLog.values[0,3:])]
G = cleanRowNorm[4]
IP = cleanRowNorm[10]
cleanRowNorm2 = cleanRowNorm[:4]+cleanRowNorm[5:10]+cleanRowNorm[11:]
cleanRowNorm3 = [x/G for x in cleanRowNorm2[:3]] +[cleanRowNorm2[3]]+ [x/G for x in cleanRowNorm2[4:9]]+[x/IP for x in cleanRowNorm2[9:20]]+[cleanRowNorm2[20]/G]+[x for x in cleanRowNorm2[21:]]

In [7]:
len(cleanRowNorm2) ==len(cleanRowNorm3)

True

In [8]:
def MakeStdRow(GameLog):
    try:
        GameLog=GameLog.drop(['GS_x','GSv2'], axis=1)
    except:
        print("Didn't drop GS_x and GSv2")
    cleanRowNorm = [x for x in map(cleanGL,GameLog.values[0,3:])]
    G = cleanRowNorm[4]
    IP = cleanRowNorm[10]
    cleanRowNorm2 = cleanRowNorm[:4]+cleanRowNorm[5:10]+cleanRowNorm[11:]
    cleanRowNorm3 = [x/G for x in cleanRowNorm2[:3]] +[cleanRowNorm2[3]]+ [x/G for x in cleanRowNorm2[4:9]]+[x/IP for x in cleanRowNorm2[9:20]]+[cleanRowNorm2[20]/G]+[x for x in cleanRowNorm2[21:]]
    return cleanRowNorm3

In [9]:
CleanRow = MakeStdRow(GameLog)
CleanRow

[0.0,
 0.05142857142857143,
 0.05142857142857143,
 3.1,
 0.0,
 0.0,
 0.005714285714285714,
 0.14285714285714285,
 0.037142857142857144,
 4.084033613445378,
 0.865546218487395,
 0.38375350140056025,
 0.3445378151260504,
 0.10644257703081232,
 0.28851540616246496,
 0.036414565826330535,
 0.0196078431372549,
 0.028011204481792718,
 0.0,
 0.8067226890756303,
 5.121214285714285,
 7.26,
 2.6,
 2.8,
 0.96,
 19.8,
 7.1,
 12.7,
 0.22899999999999998,
 1.15,
 0.265,
 77.1,
 82.0,
 100.0,
 3.82]

In [10]:
SplitLogHand.head()

,Season,Handedness,IP,ERA,TBF,H,2B,3B,R,ER,...,HR/9,K%,BB%,K-BB%,AVG_y,WHIP,BABIP,LOB%,FIP,xFIP
0,Total,vs L,124.2,- - -,541.0,113.0,13.0,4.0,50.0,48.0,...,1.16,17.7,11.5,6.3,- - -,1.40,0.265,82.0,4.77,4.63
1,Total,vs R,232.0,- - -,917.0,196.0,48.0,3.0,87.0,75.0,...,0.85,20.9,4.5,16.5,- - -,1.02,0.265,73.5,3.31,3.66


In [11]:
IPL = SplitLogHand.values[0,2]
cleanRowSplitvL = [x/IPL for x in map(cleanGL, SplitLogHand.values[0,4:11])]+[x for x in map(cleanGL, SplitLogHand.values[0,11:26])]+[x for x in map(cleanGL, SplitLogHand.values[0,27:])]

In [12]:
IPR = SplitLogHand.values[1,2]
cleanRowSplitvR = [x/IPR for x in map(cleanGL, SplitLogHand.values[1,4:11])]+[x for x in map(cleanGL, SplitLogHand.values[1,11:26])]+[x for x in map(cleanGL, SplitLogHand.values[1,27:])]

In [13]:
cleanRowSplitvL

[4.355877616747182,
 0.9098228663446054,
 0.10466988727858292,
 0.0322061191626409,
 0.4025764895330113,
 0.3864734299516908,
 0.1288244766505636,
 62.0,
 9.0,
 1.0,
 96.0,
 0.23600000000000002,
 0.327,
 0.386,
 0.308,
 6.93,
 4.48,
 1.55,
 1.16,
 17.7,
 11.5,
 6.3,
 1.4,
 0.265,
 82.0,
 4.77,
 4.63]

In [14]:
cleanRowSplitvR

[3.9525862068965516,
 0.8448275862068966,
 0.20689655172413793,
 0.01293103448275862,
 0.375,
 0.3232758620689655,
 0.09482758620689655,
 41.0,
 4.0,
 6.0,
 192.0,
 0.225,
 0.267,
 0.37,
 0.27399999999999997,
 7.45,
 1.59,
 4.68,
 0.85,
 20.9,
 4.5,
 16.5,
 1.02,
 0.265,
 73.5,
 3.31,
 3.66]

In [15]:
def MakeHandRow(SplitLogHand):
    IPL = SplitLogHand.values[0,2]
    cleanRowSplitvL = [x/IPL for x in map(cleanGL, SplitLogHand.values[0,4:11])]+[x for x in map(cleanGL, SplitLogHand.values[0,11:26])]+[x for x in map(cleanGL, SplitLogHand.values[0,27:])]
    IPR = SplitLogHand.values[1,2]
    cleanRowSplitvR = [x/IPR for x in map(cleanGL, SplitLogHand.values[1,4:11])]+[x for x in map(cleanGL, SplitLogHand.values[1,11:26])]+[x for x in map(cleanGL, SplitLogHand.values[1,27:])]
    return cleanRowSplitvL + cleanRowSplitvR

In [16]:
MakeHandRow(SplitLogHand)

[4.355877616747182,
 0.9098228663446054,
 0.10466988727858292,
 0.0322061191626409,
 0.4025764895330113,
 0.3864734299516908,
 0.1288244766505636,
 62.0,
 9.0,
 1.0,
 96.0,
 0.23600000000000002,
 0.327,
 0.386,
 0.308,
 6.93,
 4.48,
 1.55,
 1.16,
 17.7,
 11.5,
 6.3,
 1.4,
 0.265,
 82.0,
 4.77,
 4.63,
 3.9525862068965516,
 0.8448275862068966,
 0.20689655172413793,
 0.01293103448275862,
 0.375,
 0.3232758620689655,
 0.09482758620689655,
 41.0,
 4.0,
 6.0,
 192.0,
 0.225,
 0.267,
 0.37,
 0.27399999999999997,
 7.45,
 1.59,
 4.68,
 0.85,
 20.9,
 4.5,
 16.5,
 1.02,
 0.265,
 73.5,
 3.31,
 3.66]